In [1]:

import pandas as pd
import os
import statsmodels.api as sm
import numpy as np
from dotenv import load_dotenv

In [2]:
load_dotenv()
data_dir = os.getenv("DATA_PATH")
folder_path = os.path.join(data_dir, "Microsimulations")

In [3]:
# Define input and output paths
input_base_path = os.path.join(folder_path, "household")
output_base_path = os.path.join(folder_path, "with_chn")

# Ensure output directory exists
os.makedirs(output_base_path, exist_ok=True)

# Dictionary to store updated data
census_data = {}

# Loop through the years 2022 to 2030
for year in range(2022, 2031):
    input_file_path = os.path.join(input_base_path, f"census{year}_household.csv")
    output_file_path = os.path.join(output_base_path, f"census{year}_household_chn.csv")
    
    if os.path.exists(input_file_path):
        # Load the data
        census_df = pd.read_csv(input_file_path)
        
        # Initialize CHN column to 0
        census_df['chn'] = 0

        # Define housing issue conditions
        housing_issue = (
            (census_df['SHELCO'] * 12 / census_df['totalincome'] > 0.30) |  # Unaffordable
            (census_df['NOS'] == 0) |  # Unsuitable
            (census_df['REPAIR'] == 3)  # Inadequate
        )

        # Define market unaffordability condition
        market_unaffordable = (census_df['mmr']) * 12 > 0.30 * census_df['totalincome']

        # Update CHN variable
        census_df.loc[
            housing_issue & market_unaffordable &
            ~((census_df['student_household'] == 1) & (census_df['non_family_household'] == 1)),
            'chn'
        ] = 1

        # Now create stir and alt_stir after chn is assigned
       # census_df['stir'] = census_df['SHELCO'] * 12 / census_df['totalincome']
        #census_df['alt_stir'] = (census_df['mmr']) * 12 / census_df['totalincome']


        # Update CHN: Exclude individuals with STIR >= 1
        census_df.loc[census_df["stir"] >= 1, "chn"] = 0
        

        # Define deep core housing issue condition (using 50% income threshold)
        deep_housing_issue = (
            (census_df['SHELCO'] * 12 / census_df['totalincome'] > 0.50) |  # Deeply Unaffordable
            (census_df['NOS'] == 0) |  # Unsuitable
            (census_df['REPAIR'] == 3)  # Inadequate
        )

        # Define deep market unaffordability condition (50% threshold)
        deep_market_unaffordable = (census_df['mmr']) * 12 > 0.50 * census_df['totalincome']

        # Initialize dchn column to 0
        census_df['dchn'] = 0

        # Update dchn variable
        census_df.loc[
            deep_housing_issue & deep_market_unaffordable &
            ~((census_df['student_household'] == 1) & (census_df['non_family_household'] == 1)),
            'dchn'
        ] = 1

        # Update DCHN: Exclude individuals with STIR >= 1
        census_df.loc[census_df["stir"] >= 1, "dchn"] = 0

        # Export updated data
        census_df.to_csv(output_file_path, index=False)
        
        print(f"Updated CHN values and added stir/alt_stir for {year}")

    else:
        print(f"File not found: {input_file_path}")

Updated CHN values and added stir/alt_stir for 2022
Updated CHN values and added stir/alt_stir for 2023
Updated CHN values and added stir/alt_stir for 2024
Updated CHN values and added stir/alt_stir for 2025
Updated CHN values and added stir/alt_stir for 2026
Updated CHN values and added stir/alt_stir for 2027
Updated CHN values and added stir/alt_stir for 2028
Updated CHN values and added stir/alt_stir for 2029
Updated CHN values and added stir/alt_stir for 2030


In [4]:
#add 2021 census file to folder

# Define input and output paths
input_base_path = os.path.join(folder_path, "household")
output_base_path = os.path.join(folder_path, "with_chn")

# File name
input_file_name = "census2021_household.csv"
output_file_name = "census2021_household_chn.csv"

# Full paths
input_file_path = os.path.join(input_base_path, input_file_name)
output_file_path = os.path.join(output_base_path, output_file_name)

# Read the file
df = pd.read_csv(input_file_path)


  # Now create stir and alt_stir after chn is assigned
df['stir'] = df['SHELCO'] * 12 / df['totalincome']
df['alt_stir'] = (df['mmr']) * 12 / df['totalincome']

#net income share
df['netshare'] = (
    df['TOTINC_AT'] / df['totalincome']
).clip(upper=1.0)

#net income
df['netinc'] = df['totalincome'] * df['netshare']

# Save the modified dataframe
df.to_csv(output_file_path, index=False)

print(f"File saved to: {output_file_path}")

File saved to: C:/Users/mgordon/OneDrive - Financial Accountability Office of Ontario/FA2404 Housing and Homelessness Update/Data\Microsimulations\with_chn\census2021_household_chn.csv


In [5]:

# Define the path for processed files
output_base_path = os.path.join(folder_path, "with_chn")

# Dictionaries to store weighted household counts
chn_weighted_counts = {}
dchn_weighted_counts = {}

# Loop through years 2022 to 2030
for year in range(2021, 2031):
    file_path = os.path.join(output_base_path, f"census{year}_household_chn.csv")

    if os.path.exists(file_path):
        # Load the data
        census_df = pd.read_csv(file_path)

        # Check if required columns exist
        required_columns = {'chn', 'dchn', 'WEIGHT', 'HCORENEED_IND'}
        missing_columns = required_columns - set(census_df.columns)

        if missing_columns:
            print(f"Skipping {year} due to missing columns: {missing_columns}")
            continue  # Skip processing this file

        # Exclude households where HCORENEED_IND == 888
        filtered_df = census_df[census_df['HCORENEED_IND'] != 888]

        # Calculate weighted count of households where chn == 1
        chn_weight = filtered_df.loc[filtered_df['chn'] == 1, 'WEIGHT'].sum()
        chn_weighted_counts[year] = chn_weight

        # Calculate weighted count of households where dchn == 1
        dchn_weight = filtered_df.loc[filtered_df['dchn'] == 1, 'WEIGHT'].sum()
        dchn_weighted_counts[year] = dchn_weight

        print(f"[{year}] CHN weighted count: {chn_weight}, DCHN weighted count: {dchn_weight}")

    else:
        print(f"File not found: {file_path}")

print("\nTotal household weights per year:")

# Print the sum of WEIGHT for each year
for year in range(2021, 2031):
    file_path = os.path.join(output_base_path, f"census{year}_household_chn.csv")
    if os.path.exists(file_path):
        census_df = pd.read_csv(file_path)
        print(f"Sum of WEIGHT for {year}: {census_df['WEIGHT'].sum()}")


[2021] CHN weighted count: 684245.5791455524, DCHN weighted count: 154141.0278820697
[2022] CHN weighted count: 834715.6728343854, DCHN weighted count: 230270.94358313503
[2023] CHN weighted count: 832121.4039748383, DCHN weighted count: 227946.11747487643
[2024] CHN weighted count: 873264.2767260901, DCHN weighted count: 244929.585172378
[2025] CHN weighted count: 937705.9373595526, DCHN weighted count: 267898.7447957105
[2026] CHN weighted count: 967232.7473527336, DCHN weighted count: 280889.29216062214
[2027] CHN weighted count: 978742.445329859, DCHN weighted count: 287822.48006832687
[2028] CHN weighted count: 988714.1026652578, DCHN weighted count: 292470.2648668414
[2029] CHN weighted count: 1002976.269444293, DCHN weighted count: 299798.03430102044
[2030] CHN weighted count: 1022613.141810716, DCHN weighted count: 309050.7101233577

Total household weights per year:
Sum of WEIGHT for 2021: 3310969.362428404
Sum of WEIGHT for 2022: 3395297.595946454
Sum of WEIGHT for 2023: 3505

In [6]:

# Define year to process
year = 2021
file_path = os.path.join(folder_path, f"census{year}.csv")

# Check if file exists before proceeding
if os.path.exists(file_path):
    df = pd.read_csv(file_path)
    
    # Create jobless variable
    df["jobless"] = df["LFACT"].between(3, 10).astype(int)
    
    # Calculate share of records with jobless == 1 for each AGEGRP and IMMSTAT
    summary = df.groupby(["AGEGRP", df["IMMSTAT"].apply(lambda x: "IMMSTAT_3" if x == 3 else "IMMSTAT_not_3")])["jobless"].mean().reset_index()
    
    # Save summary to a CSV file
    output_path = os.path.join(folder_path, f"census_share_{year}.csv")
    summary.to_csv(output_path, index=False)


In [7]:

# Load netshare from 2021
input_base_path = os.path.join(folder_path, "with_chn")
census2021_household = pd.read_csv(os.path.join(input_base_path, "census2021_household_chn.csv"))
netshare_2021 = (census2021_household['TOTINC_AT'] / census2021_household['totalincome']).clip(upper=1.0)

# Apply to years 2022–2030
input_base_path = os.path.join(folder_path, "with_chn")

for year in range(2022, 2031):
    file_path = os.path.join(input_base_path, f"census{year}_household_chn.csv")

    if os.path.exists(file_path):
        census_df = pd.read_csv(file_path)

        # Assign netshare from 2021 (assumes same order/row count)
        census_df['netshare'] = netshare_2021.values
        census_df['netinc'] = census_df['totalincome'] * census_df['netshare']

        # Save updated file
        census_df.to_csv(file_path, index=False)
        print(f"✅ Updated netshare and netinc for {year}")
    else:
        print(f"❌ File not found for {year}: {file_path}")

✅ Updated netshare and netinc for 2022
✅ Updated netshare and netinc for 2023
✅ Updated netshare and netinc for 2024
✅ Updated netshare and netinc for 2025
✅ Updated netshare and netinc for 2026
✅ Updated netshare and netinc for 2027
✅ Updated netshare and netinc for 2028
✅ Updated netshare and netinc for 2029
✅ Updated netshare and netinc for 2030


In [8]:

input_base_path = os.path.join(folder_path, "with_chn")

for year in range(2021, 2031):
    file_path = os.path.join(input_base_path, f"census{year}_household_chn.csv")

    if os.path.exists(file_path):
        census_df = pd.read_csv(file_path)

        # Default gap to 0.0
        census_df['gap'] = 0.0

        # Only calculate gap where CHN = 1
        chn_condition = census_df['chn'] == 1

        # Use SHELCO if NOS == 1, REPAIR != 3, and SHELCO < mmr 
        use_shelco = (
            chn_condition &
            (census_df['NOS'] == 1) &
            (census_df['REPAIR'] != 3) &
            (census_df['SHELCO'] < census_df['mmr'])
        )

        # Use AMR (mmr) otherwise
        use_amr = chn_condition & ~use_shelco  # CHN == 1 but doesn't meet SHELCO condition

        # Apply SHELCO-based gap
        census_df.loc[use_shelco, 'gap'] = (
            census_df.loc[use_shelco, 'SHELCO'] * 12 - 0.3 * census_df.loc[use_shelco, 'totalincome']
        )

        # Apply AMR-based gap
        census_df.loc[use_amr, 'gap'] = (
            (census_df.loc[use_amr, 'mmr']) * 12 - 0.3 * census_df.loc[use_amr, 'totalincome']
        )

        # Save updated file
        census_df.to_csv(file_path, index=False)
        print(f"✅ Calculated gap for {year} using updated CHN conditions")
    else:
        print(f"❌ File not found for {year}: {file_path}")

✅ Calculated gap for 2021 using updated CHN conditions
✅ Calculated gap for 2022 using updated CHN conditions
✅ Calculated gap for 2023 using updated CHN conditions
✅ Calculated gap for 2024 using updated CHN conditions
✅ Calculated gap for 2025 using updated CHN conditions
✅ Calculated gap for 2026 using updated CHN conditions
✅ Calculated gap for 2027 using updated CHN conditions
✅ Calculated gap for 2028 using updated CHN conditions
✅ Calculated gap for 2029 using updated CHN conditions
✅ Calculated gap for 2030 using updated CHN conditions


In [9]:
# After assigning gaps
print(f"{year}: CHN=1 count: {census_df['chn'].sum()}, GAP > 0 count: {(census_df['gap'] > 0).sum()}")


2030: CHN=1 count: 8741, GAP > 0 count: 8741


In [10]:
import os
import pandas as pd

# Load NPR household IDs from CSV
npr_hh_path = os.path.join(folder_path, "npr_household_ids.csv")
npr_hh_df = pd.read_csv(npr_hh_path)
npr_household_ids = npr_hh_df['HH_ID'].tolist()

input_base_path = os.path.join(folder_path, "with_chn")

for year in range(2021, 2031):  # 2021 to 2030 inclusive
    file_path = os.path.join(input_base_path, f"census{year}_household_chn.csv")

    if os.path.exists(file_path):
        census_df = pd.read_csv(file_path)

        # ✅ Add nprhh column based on imported NPR household IDs
        if 'HH_ID' in census_df.columns:
            census_df['nprhh'] = census_df['HH_ID'].isin(npr_household_ids).astype(int)
        else:
            print(f"⚠️ HH_ID column not found in {year} dataset.")
            census_df['nprhh'] = 0

        # Initialize COHB to 0.0
        census_df['cohb'] = 0.0

        # Condition: renter, in core housing need, stir > 0.3
        condition = (
            (census_df['TENUR'] == 2) &
            (census_df['chn'] == 1) &
            (census_df['stir'] > 0.3)
        )

        # Pre-calculate COHB components
        mmr_80 = 0.8 * 12 * census_df.loc[condition, 'mmr']
        shelco_100_capped = (12 * census_df.loc[condition, 'SHELCO']).clip(
            upper=(12 * census_df.loc[condition, 'mmr'])
        )
        eligible_cost = pd.concat([mmr_80, shelco_100_capped], axis=1).max(axis=1)

        netinc_30 = 0.3 * census_df.loc[condition, 'netinc']
        cohb_values = eligible_cost - netinc_30

        # Final COHB assignment with clipping
        census_df.loc[condition, 'cohb'] = cohb_values.clip(lower=0)

        # 🔢 Filter: nprhh == 1, chn == 1, and exclude HCORENEED_IND == 888
        filter_condition = (
            (census_df['nprhh'] == 1) &
            (census_df['chn'] == 1) &
            (census_df['HCORENEED_IND'] != 888)
        )

        if 'WEIGHT' in census_df.columns:
            weighted_count = census_df.loc[filter_condition, 'WEIGHT'].sum()
            print(f"📅 {year}: Weighted households with nprhh == 1, chn == 1, HCORENEED_IND != 888: {weighted_count:,.0f}")
        else:
            print(f"⚠️ WEIGHT column missing in {year} data.")

        # Save updated file
        census_df.to_csv(file_path, index=False)
        print(f"✅ Finished processing {year}")

    else:
        print(f"❌ File not found for {year}: {file_path}")


📅 2021: Weighted households with nprhh == 1, chn == 1, HCORENEED_IND != 888: 27,916
✅ Finished processing 2021
📅 2022: Weighted households with nprhh == 1, chn == 1, HCORENEED_IND != 888: 34,932
✅ Finished processing 2022
📅 2023: Weighted households with nprhh == 1, chn == 1, HCORENEED_IND != 888: 48,281
✅ Finished processing 2023
📅 2024: Weighted households with nprhh == 1, chn == 1, HCORENEED_IND != 888: 63,627
✅ Finished processing 2024
📅 2025: Weighted households with nprhh == 1, chn == 1, HCORENEED_IND != 888: 66,205
✅ Finished processing 2025
📅 2026: Weighted households with nprhh == 1, chn == 1, HCORENEED_IND != 888: 63,775
✅ Finished processing 2026
📅 2027: Weighted households with nprhh == 1, chn == 1, HCORENEED_IND != 888: 59,948
✅ Finished processing 2027
📅 2028: Weighted households with nprhh == 1, chn == 1, HCORENEED_IND != 888: 55,839
✅ Finished processing 2028
📅 2029: Weighted households with nprhh == 1, chn == 1, HCORENEED_IND != 888: 54,413
✅ Finished processing 2029
📅

In [ ]:
import os
import pandas as pd

mean_cohb = {}
mean_gap = {}
mean_income = {}  # NEW dictionary to store weighted avg income

for year in range(2021, 2031):
    file_path = os.path.join(input_base_path, f"census{year}_household_chn.csv")

    if os.path.exists(file_path):
        census_df_full = pd.read_csv(file_path)  # full, unfiltered

        # Ensure the necessary columns exist
        if all(col in census_df_full.columns for col in ['cohb', 'gap', 'WEIGHT', 'HCORENEED_IND', 'totalincome']):
            
            # 1️⃣ For COHB & GAP → exclude HCORENEED_IND == 888
            census_df = census_df_full[census_df_full['HCORENEED_IND'] != 888].copy()

            # COHB > 0
            cohb_positive = census_df[census_df['cohb'] > 0]
            if not cohb_positive.empty:
                weighted_mean_cohb = (cohb_positive['cohb'] * cohb_positive['WEIGHT']).sum() / cohb_positive['WEIGHT'].sum()
                mean_cohb[year] = weighted_mean_cohb
            else:
                mean_cohb[year] = 0

            # GAP > 0
            gap_positive = census_df[census_df['gap'] > 0]
            if not gap_positive.empty:
                weighted_mean_gap = (gap_positive['gap'] * gap_positive['WEIGHT']).sum() / gap_positive['WEIGHT'].sum()
                mean_gap[year] = weighted_mean_gap
            else:
                mean_gap[year] = 0

            # 2️⃣ For Income → use *full*, unfiltered dataframe
            if not census_df_full.empty:
                weighted_mean_income = (census_df_full['totalincome'] * census_df_full['WEIGHT']).sum() / census_df_full['WEIGHT'].sum()
                mean_income[year] = weighted_mean_income
            else:
                mean_income[year] = 0
        else:
            print(f"❌ Missing columns in {year}, skipping.")
    else:
        print(f"❌ File not found for {year}")

# ✅ Print results
print("\n📊 Weighted Mean COHB (for values > 0, excluding HCORENEED_IND == 888):")
for year, val in mean_cohb.items():
    print(f"{year}: {val:.2f}")

print("\n📊 Weighted Mean GAP (for values > 0, excluding HCORENEED_IND == 888):")
for year, val in mean_gap.items():
    print(f"{year}: {val:.2f}")

print("\n📊 Weighted Mean Income (ALL households, INCLUDING HCORENEED_IND == 888):")
for year, val in mean_income.items():
    print(f"{year}: ${val:,.2f}")



📊 Weighted Mean COHB (for values > 0, excluding HCORENEED_IND == 888):
2021: 4401.67
2022: 5215.27
2023: 5525.17
2024: 5963.06
2025: 6514.40
2026: 6815.26
2027: 6996.40
2028: 7166.57
2029: 7368.64
2030: 7575.98

📊 Weighted Mean GAP (for values > 0, excluding HCORENEED_IND == 888):
2021: 3783.17
2022: 4227.66
2023: 4544.06
2024: 4870.02
2025: 5198.68
2026: 5405.40
2027: 5550.51
2028: 5691.48
2029: 5857.36
2030: 6022.70

📊 Weighted Mean Income (ALL households, INCLUDING HCORENEED_IND == 888):
2021: $59,184.27
2022: $59,723.49
2023: $63,791.09
2024: $66,897.60
2025: $68,827.73
2026: $70,693.85
2027: $72,832.27
2028: $75,048.70
2029: $77,006.82
2030: $78,845.55


In [12]:
#trace file


# Folder path
base_path = os.path.join(folder_path, "with_chn")

# Years to process
years = range(2021, 2031)

# Row number to extract (0-based index)
target_row = 51  # Change this to any row index you want

# List to store selected rows
selected_rows = []

for year in years:
    file_name = f"census{year}_household_chn.csv"
    file_path = os.path.join(base_path, file_name)
    
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        if len(df) > target_row:
            selected_row = df.iloc[target_row]
            selected_rows.append(selected_row)
        else:
            print(f"File {file_name} has less than {target_row + 1} rows.")
    else:
        print(f"File not found: {file_path}")

# Combine and save to chn_trace.csv
if selected_rows:
    chn_trace_df = pd.DataFrame(selected_rows)
    output_path = os.path.join(base_path, f"chn_trace_row{target_row + 2}.csv")
    chn_trace_df.to_csv(output_path, index=False)
    print(f"chn_trace_row{target_row + 1}.csv created at {output_path}")
else:
    print("No data found to create trace file.")

chn_trace_row52.csv created at C:/Users/mgordon/OneDrive - Financial Accountability Office of Ontario/FA2404 Housing and Homelessness Update/Data\Microsimulations\with_chn\chn_trace_row53.csv


In [13]:


# Load the data
df = pd.read_csv(os.path.join(folder_path, "with_chn", "census2021_household_chn.csv"))

# Check required columns
required_cols = ['netinc', 'totalincome', 'WEIGHT', 'quintile']
if all(col in df.columns for col in required_cols):
    # Compute netshare safely
    df['netshare'] = df['netinc'] / df['totalincome']
    df = df.replace([float('inf'), -float('inf')], pd.NA).dropna(subset=['netshare'])

    # Group by quintile and calculate weighted average netshare
    summary = (
        df.groupby('quintile')
        .apply(lambda g: (g['netshare'] * g['WEIGHT']).sum() / g['WEIGHT'].sum())
        .reset_index(name='weighted_netshare')
    )

    # Format output
    summary['weighted_netshare'] = summary['weighted_netshare'].round(4)
    print("\n✅ Weighted Average Netshare by Quintile:\n")
    print(summary)
else:
    print("❌ Missing required columns: netinc, totalincome, WEIGHT, or quintile.")



✅ Weighted Average Netshare by Quintile:

   quintile  weighted_netshare
0         1             0.9442
1         2             0.9152
2         3             0.8822


C:\Users\mgordon\AppData\Local\Temp\ipykernel_23268\510544637.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (g['netshare'] * g['WEIGHT']).sum() / g['WEIGHT'].sum())


In [14]:
print(df[['totalincome', 'netinc', 'netshare']].sort_values(by='netshare').head(10))


       totalincome   netinc   netshare
5860           401  -7000.0 -17.456359
30378         1000  -6000.0  -6.000000
25865         4400 -23000.0  -5.227273
8073         11300 -50000.0  -4.424779
5262          1000  -4000.0  -4.000000
16587        13400 -53000.0  -3.955224
24238          801  -3000.0  -3.745318
9991          6800 -20000.0  -2.941176
7770         11900 -30000.0  -2.521008
4357          4500 -11000.0  -2.444444


Ignore for now: COHB/affordable housing program analysis

In [15]:

import pandas as pd
output_path = os.path.join(folder_path, "with_chn", "subset_2024.csv")
# Example: assuming df2024 is loaded
df = pd.read_csv(os.path.join(folder_path, "with_chn", "census2024_household_chn.csv"))

df = df[
    (df['TENUR'] == 2) &
    (df['chn'] == 1) &
    (df['SUBSIDY'] == 0)
].copy()




def select_households_below_weighted_avg(df, target_avg=20000):
    df_sorted = df.sort_values(by="totalincome").reset_index(drop=True)
    
    df_sorted['cum_weighted_income'] = (df_sorted['totalincome'] * df_sorted['WEIGHT']).cumsum()
    df_sorted['cum_weight'] = df_sorted['WEIGHT'].cumsum()
    df_sorted['cum_weighted_avg'] = df_sorted['cum_weighted_income'] / df_sorted['cum_weight']

    # Find the row where weighted average is closest to target_avg
    df_sorted['abs_diff'] = (df_sorted['cum_weighted_avg'] - target_avg).abs()
    best_idx = df_sorted['abs_diff'].idxmin()

    subset_df = df_sorted.loc[:best_idx].copy()
    subset_df.drop(columns=['cum_weighted_income', 'cum_weight', 'cum_weighted_avg', 'abs_diff'], inplace=True)

    return subset_df



subset_df = select_households_below_weighted_avg(df)

subset_df['estgap'] = 12 * subset_df['mmr'] - 0.3 * subset_df['totalincome']
subset_df['ntgap'] = 0.8 * 12 * subset_df['mmr'] - 0.3 * subset_df['totalincome']


weighted_avg = (subset_df['totalincome'] * subset_df['WEIGHT']).sum() / subset_df['WEIGHT'].sum()
print(f"Weighted average income: ${weighted_avg:,.2f}")
subset_df.to_csv(output_path, index=False)


Weighted average income: $20,000.85


In [25]:
import os
import pandas as pd
import numpy as np

df = pd.read_csv(os.path.join(folder_path, "with_chn", "subset_2024.csv"))

target_weight = 22000
target_avg = 10600

# Split below/above target
below = df[df["cohb"] <= target_avg].copy()
above = df[df["cohb"] >  target_avg].copy()

# Sort: below from highest up to target, above from lowest down to target
below = below.sort_values("cohb", ascending=False).reset_index(drop=True)
above = above.sort_values("cohb", ascending=True).reset_index(drop=True)

selected = []
W = 0.0
WC = 0.0

i = j = 0
while W < target_weight and (i < len(below) or j < len(above)):
    # current average if we add a candidate
    def score(row):
        w = row["WEIGHT"]
        c = row["cohb"]
        W_new = min(target_weight, W + w)
        w_use = W_new - W
        avg_new = (WC + c * w_use) / W_new
        return abs(avg_new - target_avg)

    candidates = []
    if i < len(below): candidates.append(("below", below.iloc[i]))
    if j < len(above): candidates.append(("above", above.iloc[j]))

    # choose the candidate that gets the running average closest to target
    side, row = min(candidates, key=lambda t: score(t[1]))

    w = row["WEIGHT"]
    c = row["cohb"]

    # take full weight or trim last
    take = min(w, target_weight - W)

    row_copy = row.copy()
    row_copy["WEIGHT"] = take
    selected.append(row_copy)

    W += take
    WC += c * take

    if side == "below": i += 1
    else: j += 1

final_avg = WC / W
print(f"Final weighted avg COHB: {final_avg:,.2f} (target {target_avg:,.2f}), total weight {W:,.0f}")

final_df = pd.DataFrame(selected)
final_df.to_csv(os.path.join(folder_path, "with_chn", "final_subset_2024_balanced.csv"), index=False)


Final weighted avg COHB: 10,591.33 (target 10,600.00), total weight 22,000


In [26]:
import os
import pandas as pd

# Load your balanced 2024 selection (this includes trimmed WEIGHT)
final_subset = pd.read_csv(os.path.join(folder_path, "with_chn", "final_subset_2024_balanced.csv"))

# Create a lookup: HH_ID -> selected_weight_2024
sel_w = final_subset.groupby("HH_ID", as_index=False)["WEIGHT"].sum()
sel_w = sel_w.rename(columns={"WEIGHT": "SEL_WEIGHT_2024"})

years = range(2024, 2031)

for year in years:
    df_year = pd.read_csv(os.path.join(folder_path, "with_chn", f"census{year}_household_chn.csv"))

    # Join to keep only selected HH_IDs and bring in the selected weights
    df_matched = df_year.merge(sel_w, on="HH_ID", how="inner")

    if df_matched.empty:
        print(f"⚠️ No matching HH_IDs found in {year} data.")
        continue

    weighted_avg_cohb = (df_matched["cohb"] * df_matched["SEL_WEIGHT_2024"]).sum() / df_matched["SEL_WEIGHT_2024"].sum()

    print(f"✅ {year}: Weighted avg COHB = ${weighted_avg_cohb:,.2f} "
          f"over {df_matched['SEL_WEIGHT_2024'].sum():,.0f} fixed-weight households")


✅ 2024: Weighted avg COHB = $10,591.33 over 22,000 fixed-weight households
✅ 2025: Weighted avg COHB = $11,379.10 over 22,000 fixed-weight households
✅ 2026: Weighted avg COHB = $11,842.16 over 22,000 fixed-weight households
✅ 2027: Weighted avg COHB = $12,163.57 over 22,000 fixed-weight households
✅ 2028: Weighted avg COHB = $12,471.81 over 22,000 fixed-weight households
✅ 2029: Weighted avg COHB = $12,796.03 over 22,000 fixed-weight households
✅ 2030: Weighted avg COHB = $13,130.93 over 22,000 fixed-weight households


In [18]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv(os.path.join(folder_path, "with_chn", "census2023_household_chn.csv"))


# Define conditions based on bedsuit values
conditions = [
    (df['bedsuit'] == 0) & (df['totalincome'] <= 40000),
    (df['bedsuit'] == 1) & (df['totalincome'] <= 49000),
    (df['bedsuit'] == 2) & (df['totalincome'] <= 56000),
    (df['bedsuit'] == 3) & (df['totalincome'] <= 62000),
    (df['bedsuit'] >= 4) & (df['totalincome'] <= 75000),
]

# Combine all conditions using logical OR
combined_condition = conditions[0]
for cond in conditions[1:]:
    combined_condition |= cond

# Create the filtered subset
subset_df = df[combined_condition].copy()


In [19]:
# Filter for households in core housing need and valid HCORENEED_IND
chn_df = subset_df[(subset_df['chn'] == 1) & (subset_df['HCORENEED_IND'] != 888)]

# Group by TENUR and sum the WEIGHT for each group
weighted_chn_by_tenur = chn_df.groupby('TENUR')['WEIGHT'].sum()

# Display the result
print(weighted_chn_by_tenur)


TENUR
1.0    229540.773505
2.0    424351.093382
Name: WEIGHT, dtype: float64


calculate CNIT to use for asset threshold shares

In [20]:
import pandas as pd

# Step 1: Load the data
df = pd.read_csv(os.path.join(folder_path, "with_chn", "census2023_household_chn.csv"))

# Step 2: Create a filtered copy for households in core housing need with valid HCORENEED_IND
df_copy = df[(df['chn'] == 1) & (df['HCORENEED_IND'] != 888)].copy()

# Step 3: Create the cnit variable
df_copy['cnit'] = (12 * df_copy['mmr']) / 0.3

# Step 4: Calculate weighted average CNIT by bedsuit
weighted_avg_cnit = (
    df_copy
    .groupby('bedsuit')
    .apply(lambda g: (g['cnit'] * g['WEIGHT']).sum() / g['WEIGHT'].sum())
)

# Display the result
print(weighted_avg_cnit)


# Filter households with bedsuit >= 4
df_4plus = df_copy[df_copy['bedsuit'] >= 4]

# Calculate weighted average CNIT
weighted_avg_cnit_4plus = (df_4plus['cnit'] * df_4plus['WEIGHT']).sum() / df_4plus['WEIGHT'].sum()

# Display the result
print(f"Weighted average CNIT for 4+ bedroom-suitable households: {weighted_avg_cnit_4plus:.2f}")



bedsuit
0.0     46656.115253
1.0     57890.077750
2.0     68351.707704
3.0     82669.044239
4.0    105104.427040
5.0    103770.710224
dtype: float64
Weighted average CNIT for 4+ bedroom-suitable households: 104928.78


C:\Users\mgordon\AppData\Local\Temp\ipykernel_23268\413785005.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (g['cnit'] * g['WEIGHT']).sum() / g['WEIGHT'].sum())


In [21]:
import pandas as pd
import os

# Path to your directory
base_path = os.path.join(folder_path, "with_chn")

# Loop through years 2023 to 2030
for year in range(2023, 2031):
    file_path = os.path.join(base_path, f"census{year}_household_chn.csv")
    
    # Load the data
    df = pd.read_csv(file_path)
    
    # Filter for households in core housing need with valid HCORENEED_IND
    df_copy = df[(df['chn'] == 1) & (df['HCORENEED_IND'] != 888)].copy()
    
    # Create the cnit variable
    df_copy['cnit'] = (12 * df_copy['mmr']) / 0.3
    
    # Calculate weighted average CNIT by bedsuit
    weighted_avg_cnit = (
        df_copy
        .groupby('bedsuit')
        .apply(lambda g: (g['cnit'] * g['WEIGHT']).sum() / g['WEIGHT'].sum())
    )
    
    # Display the result for this year
    print(f"\nYear {year} - Weighted average CNIT by bedsuit:")
    print(weighted_avg_cnit)
    
    # Filter households with bedsuit >= 4
    df_4plus = df_copy[df_copy['bedsuit'] >= 4]
    
    # Calculate weighted average CNIT for 4+ bedroom-suitable households
    if not df_4plus.empty:
        weighted_avg_cnit_4plus = (df_4plus['cnit'] * df_4plus['WEIGHT']).sum() / df_4plus['WEIGHT'].sum()
        print(f"Weighted average CNIT for 4+ bedroom-suitable households: {weighted_avg_cnit_4plus:.2f}")
    else:
        print("No 4+ bedroom-suitable households in this year.")



Year 2023 - Weighted average CNIT by bedsuit:
bedsuit
0.0     46656.115253
1.0     57890.077750
2.0     68351.707704
3.0     82669.044239
4.0    105104.427040
5.0    103770.710224
dtype: float64
Weighted average CNIT for 4+ bedroom-suitable households: 104928.78

Year 2024 - Weighted average CNIT by bedsuit:
bedsuit
0.0     49087.216632
1.0     60928.740115
2.0     71939.638954
3.0     87240.025706
4.0    110645.412118
5.0    108870.932761
dtype: float64
Weighted average CNIT for 4+ bedroom-suitable households: 110394.67

Year 2025 - Weighted average CNIT by bedsuit:
bedsuit
0.0     51880.582507
1.0     64474.329173
2.0     76024.373526
3.0     92002.957092
4.0    116203.548621
5.0    115489.868349
dtype: float64
Weighted average CNIT for 4+ bedroom-suitable households: 116108.36


C:\Users\mgordon\AppData\Local\Temp\ipykernel_23268\3545644307.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (g['cnit'] * g['WEIGHT']).sum() / g['WEIGHT'].sum())
C:\Users\mgordon\AppData\Local\Temp\ipykernel_23268\3545644307.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (g['cnit'] * g['WEIGHT']).sum() / g['WEIGHT'].sum())
C:\Users\mgordon\AppData\Local\Temp\ip


Year 2026 - Weighted average CNIT by bedsuit:
bedsuit
0.0     53590.580776
1.0     66533.805226
2.0     78487.285655
3.0     94967.111728
4.0    119869.815996
5.0    118595.044190
dtype: float64
Weighted average CNIT for 4+ bedroom-suitable households: 119702.24


C:\Users\mgordon\AppData\Local\Temp\ipykernel_23268\3545644307.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (g['cnit'] * g['WEIGHT']).sum() / g['WEIGHT'].sum())



Year 2027 - Weighted average CNIT by bedsuit:
bedsuit
0.0     54907.662755
1.0     68150.913968
2.0     80432.010484
3.0     97312.368466
4.0    122754.689336
5.0    121794.390422
dtype: float64
Weighted average CNIT for 4+ bedroom-suitable households: 122633.33

Year 2028 - Weighted average CNIT by bedsuit:
bedsuit
0.0     56197.159451
1.0     69582.405457
2.0     82322.983444
3.0     99627.666024
4.0    125669.450164
5.0    124957.636856
dtype: float64
Weighted average CNIT for 4+ bedroom-suitable households: 125582.92


C:\Users\mgordon\AppData\Local\Temp\ipykernel_23268\3545644307.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (g['cnit'] * g['WEIGHT']).sum() / g['WEIGHT'].sum())
C:\Users\mgordon\AppData\Local\Temp\ipykernel_23268\3545644307.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (g['cnit'] * g['WEIGHT']).sum() / g['WEIGHT'].sum())
C:\Users\mgordon\AppData\Local\Temp\ip


Year 2029 - Weighted average CNIT by bedsuit:
bedsuit
0.0     57526.632127
1.0     71222.623501
2.0     84282.792903
3.0    101921.854778
4.0    128176.740228
5.0    128017.097777
dtype: float64
Weighted average CNIT for 4+ bedroom-suitable households: 128157.77

Year 2030 - Weighted average CNIT by bedsuit:
bedsuit
0.0     58890.147376
1.0     72849.623994
2.0     86300.785775
3.0    104298.078916
4.0    131275.789583
5.0    131091.858549
dtype: float64


C:\Users\mgordon\AppData\Local\Temp\ipykernel_23268\3545644307.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (g['cnit'] * g['WEIGHT']).sum() / g['WEIGHT'].sum())


Weighted average CNIT for 4+ bedroom-suitable households: 131253.92


In [22]:
import pandas as pd

# Load the data
df = pd.read_csv(os.path.join(folder_path, "with_chn", "census2023_household_chn.csv"))

# Subset for CHN == 1, HCORENEED_IND != 888, and NPRHH == 0
filtered_df = df[(df['chn'] == 1) & (df['HCORENEED_IND'] != 888) ]
#& (df['nprhh'] == 0)
# Calculate weighted counts for each category
tenur_2 = filtered_df[filtered_df['TENUR'] == 2]['WEIGHT'].sum()
tenur_1_presmortg_0 = filtered_df[(filtered_df['TENUR'] == 1) & (filtered_df['PRESMORTG'] == 0)]['WEIGHT'].sum()
tenur_1_presmortg_1 = filtered_df[(filtered_df['TENUR'] == 1) & (filtered_df['PRESMORTG'] == 1)]['WEIGHT'].sum()


# Total weighted households (after filtering)
total_weighted = filtered_df['WEIGHT'].sum()

# Print the results
print(f"Weighted count (TENUR = 2): {tenur_2:,.0f}")
print(f"Weighted count (TENUR = 1 & PRESMORTG = 0): {tenur_1_presmortg_0:,.0f}")
print(f"Weighted count (TENUR = 1 & PRESMORTG = 1): {tenur_1_presmortg_1:,.0f}")
print(f"Total weighted households (filtered): {total_weighted:,.0f}")

Weighted count (TENUR = 2): 519,059
Weighted count (TENUR = 1 & PRESMORTG = 0): 118,615
Weighted count (TENUR = 1 & PRESMORTG = 1): 194,447
Total weighted households (filtered): 832,121


In [23]:
import os
import pandas as pd

# Path to your directory
base_path = os.path.join(folder_path, "with_chn")

# Loop through years 2023 to 2030
for year in range(2023, 2031):
    file_path = os.path.join(base_path, f"census{year}_household_chn.csv")

    if os.path.exists(file_path):
        # Load the data
        df = pd.read_csv(file_path)

        # Subset for CHN == 1, HCORENEED_IND != 888, and NPRHH == 0
        filtered_df = df[(df['chn'] == 1) & (df['HCORENEED_IND'] != 888)]

        # Calculate weighted counts
        tenur_2 = filtered_df[filtered_df['TENUR'] == 2]['WEIGHT'].sum()
        tenur_1_presmortg_0 = filtered_df[(filtered_df['TENUR'] == 1) & (filtered_df['PRESMORTG'] == 0)]['WEIGHT'].sum()
        tenur_1_presmortg_1 = filtered_df[(filtered_df['TENUR'] == 1) & (filtered_df['PRESMORTG'] == 1)]['WEIGHT'].sum()

        # Total weighted households (after filtering)
        total_weighted = filtered_df['WEIGHT'].sum()

        # Print the results for this year
        print(f"\n📅 Year {year}:")
        print(f"Weighted count (TENUR = 2): {tenur_2:,.0f}")
        print(f"Weighted count (TENUR = 1 & PRESMORTG = 0): {tenur_1_presmortg_0:,.0f}")
        print(f"Weighted count (TENUR = 1 & PRESMORTG = 1): {tenur_1_presmortg_1:,.0f}")
        print(f"Total weighted households (filtered): {total_weighted:,.0f}")

    else:
        print(f"❌ File not found for year {year}, skipping.")



📅 Year 2023:
Weighted count (TENUR = 2): 519,059
Weighted count (TENUR = 1 & PRESMORTG = 0): 118,615
Weighted count (TENUR = 1 & PRESMORTG = 1): 194,447
Total weighted households (filtered): 832,121

📅 Year 2024:
Weighted count (TENUR = 2): 550,283
Weighted count (TENUR = 1 & PRESMORTG = 0): 122,228
Weighted count (TENUR = 1 & PRESMORTG = 1): 200,753
Total weighted households (filtered): 873,264

📅 Year 2025:
Weighted count (TENUR = 2): 590,224
Weighted count (TENUR = 1 & PRESMORTG = 0): 128,936
Weighted count (TENUR = 1 & PRESMORTG = 1): 218,546
Total weighted households (filtered): 937,706

📅 Year 2026:
Weighted count (TENUR = 2): 606,784
Weighted count (TENUR = 1 & PRESMORTG = 0): 133,679
Weighted count (TENUR = 1 & PRESMORTG = 1): 226,771
Total weighted households (filtered): 967,233

📅 Year 2027:
Weighted count (TENUR = 2): 610,936
Weighted count (TENUR = 1 & PRESMORTG = 0): 137,959
Weighted count (TENUR = 1 & PRESMORTG = 1): 229,847
Total weighted households (filtered): 978,742


In [24]:
#calculate avg income growth

